In [1]:
#| hide
from nbdev.showdoc import *

In [2]:
#| default_exp readers_tests

In [1]:
#| export
import urllib
import urllib.request
import gzip
import shutil
from pathlib import Path



In [2]:
import allos
allos.__path__

['/data/analysis/data_mcandrew/Allos_new/allos_env/lib/python3.9/site-packages/allos']

In [5]:
#| export
from pathlib import Path
import os
import pandas as pd


import os
from pathlib import Path

def get_resource_path(filename):
    """Dynamically find the correct path to the 'resources/' directory based on execution location."""
    # Detect if running from the library (allos/) or a notebook (nbs/)
    if "__file__" in globals():
        # Running from the library, path is relative to the project root
        project_root = Path(__file__).resolve().parent
    else:
        # Running from a notebook, path is two levels up
        project_root = Path.cwd().resolve().parents[0]

    # Construct the full path to the resource file
    resource_path = project_root / "resources" / filename

    # Debug message for locating the file
    print(f"\n🔎 Looking for file at: {resource_path}")

    # If the file does not exist, generate an error with the directory structure
    if not resource_path.exists():
        dir_structure = f"\n📂 Directory structure of {resource_path.parent}:\n"
        
        # Traverse and collect folder structure
        for root, dirs, files in os.walk(resource_path.parent):
            level = root.replace(str(resource_path.parent), "").count(os.sep)
            indent = " " * (level * 4)
            dir_structure += f"{indent}📁 {Path(root).name}/\n"
            for f in files:
                dir_structure += f"{indent}    📄 {f}\n"

        # Raise an error including the directory structure
        raise FileNotFoundError(f"\n❌ File not found at: {resource_path}\n{dir_structure}")

    print(f"✅ File found at: {resource_path}")
    return resource_path



In [6]:

file_path = get_resource_path("e18.mouse.clusters.csv")


🔎 Looking for file at: /data/analysis/data_mcandrew/allos_2/allos/resources/e18.mouse.clusters.csv
✅ File found at: /data/analysis/data_mcandrew/allos_2/allos/resources/e18.mouse.clusters.csv


In [5]:
#| export

from pathlib import Path
import urllib.request
import gzip
import shutil

#| export
def download_test_data(
    url: str = "https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748087/suppl/GSM3748087%5F190c.isoforms.matrix.txt.gz",
    output_filename: str = None
) -> str:
    """
    Downloads a test data file from a specified URL, saves it one directory back in `../data/`, and extracts it.

    Parameters:
    url (str): The URL of the file to be downloaded. Defaults to a pre-defined test dataset.
    output_filename (str, optional): Custom name for the extracted file. Defaults to "sample_isomatrix.txt".

    Returns:
    str: The absolute path of the extracted file if successful. If the file already exists, return its path instead.
    """
    data_dir = Path("..") / "data"
    data_dir.mkdir(parents=True, exist_ok=True)

    print(f"Starting download of test data from {url}")

    # Extract filename from the URL
    filename = url.split('/')[-1]
    compressed_file = data_dir / filename

    # Determine output file path
    if output_filename:
        extracted_file = data_dir / output_filename
    else:
        extracted_file = data_dir / "sample_isomatrix.txt"

    # If the extracted file already exists, return its path
    if extracted_file.exists():
        print(f"File already exists: {extracted_file}")
        return str(extracted_file.resolve())

    # Download the file
    try:
        urllib.request.urlretrieve(url, compressed_file)
        print("File downloaded successfully")
    except Exception as e:
        print(f"Failed to download the file: {e}")
        return None

    # Extract the file
    try:
        with gzip.open(compressed_file, 'rb') as f_in:
            with open(extracted_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"File extracted successfully: {extracted_file}")
        return str(extracted_file.resolve())
    except Exception as e:
        print(f"Failed to extract the file: {e}")
        return None
    finally:
        # Clean up the compressed file
        if compressed_file.exists():
            compressed_file.unlink()



In [6]:
#| export
import os
import pandas as pd
import scanpy as sc
from anndata import AnnData

def iso_concat(data_inputs, batch_info=None, batch_type='path'):
    """
    Concatenates a list of AnnData objects or paths to AnnData objects based on the union of transcriptIds,
    while preserving geneId information which might be non-unique per transcriptId.
    Missing values are filled with zeros. Adds a batch column to `.obs` based on the file path, obs_names, or numeric.

    Parameters:
    data_inputs (list of str or AnnData):
        List of paths to AnnData objects or AnnData objects to concatenate.
    batch_info (list of str, optional):
        List of batch identifiers for each AnnData object in data_inputs.
        If not provided, batch identifiers are extracted from file paths, obs_names, or a numeric sequence.
    batch_type (str, optional):
        Specifies which type of batch information to use. One of ['path', 'obs_names', 'numeric'].
        Defaults to 'path'.

    Returns:
    AnnData:
        A single concatenated AnnData object with harmonized features, geneId annotations, and batch info.
    """
    adata_list = []
    df_list = []
    gene_ids = {}
    batch_info_list = []

    for i, data_input in enumerate(data_inputs):
        # Check if the input is a path (string) or an AnnData object
        if isinstance(data_input, str):
            adata = sc.read_h5ad(data_input)
            # Determine batch label based on batch_type
            if batch_type == 'path':
                batch = os.path.basename(data_input).split('_isomatrix')[0] if batch_info is None else batch_info[i]
            elif batch_type == 'obs_names':
                batch = adata.obs_names[0] if batch_info is None else batch_info[i]
            elif batch_type == 'numeric':
                batch = str(i)
            else:
                raise ValueError("batch_type should be 'path', 'obs_names' or 'numeric'.")
        elif isinstance(data_input, AnnData):
            adata = data_input
            # Determine batch label when input is already an AnnData object
            if batch_type == 'obs_names':
                batch = adata.obs_names[0] if batch_info is None else batch_info[i]
            elif batch_type == 'numeric':
                batch = str(i)
            else:
                raise ValueError("batch_type should be 'obs_names' or 'numeric' when passing AnnData objects.")
        else:
            raise ValueError("data_inputs must be a list of paths to AnnData objects or AnnData objects.")

        adata_list.append(adata)
        # Convert adata.X to a DataFrame for outer-join concatenation
        df = pd.DataFrame(adata.X.T, index=adata.var['transcriptId'], columns=adata.obs_names)
        df_list.append(df)
        batch_info_list.extend([batch] * adata.n_obs)

        # Map transcriptId to its geneId
        for transcript_id, gene_id in zip(adata.var['transcriptId'], adata.var['geneId']):
            gene_ids[transcript_id] = gene_id

    # Perform an outer join on all DataFrames
    concat_df = pd.concat(df_list, axis=1, join='outer').fillna(0)

    # Create a new var DataFrame with geneIds mapped from transcriptIds
    var_df = pd.DataFrame(index=concat_df.index)
    var_df['geneId'] = pd.Series(gene_ids).reindex(concat_df.index)

    # Build an AnnData object, transposing so obs are rows
    concatenated_adata = sc.AnnData(X=concat_df.T, var=var_df)

    # Add batch info to obs
    concatenated_adata.obs['batch'] = batch_info_list

    return concatenated_adata



In [7]:
#| export

import pandas as pd
import warnings
from scipy.sparse import csr_matrix
import scanpy as sc
from anndata import AnnData

def read_sicelore_isomatrix(
    file_path: str,
    gene_id_label: str = "geneId",
    transcript_id_label: str = "transcriptId",
    remove_undef: bool = True,
    sparse: bool = False
) -> AnnData:
    """
    Read a SiCeLoRe isomatrix file (tab-delimited) and convert it into a scanpy-compatible AnnData object.

    Parameters
    ----------
    file_path : str
        Path to the isomatrix file (tab-delimited).
    gene_id_label : str, optional
        Row/column label used for gene IDs (default "geneId").
    transcript_id_label : str, optional
        Row/column label used for transcript IDs (default "transcriptId").
    remove_undef : bool, optional
        Whether to remove rows with transcriptId="undef" (default True).
    sparse : bool, optional
        Whether to store the matrix in sparse format (default False).

    Returns
    -------
    anndata.AnnData
        An AnnData object containing numeric data in `.X` and metadata in `.var`.
    """
    # Read in the file, expecting rows to be features initially
    df = pd.read_csv(file_path, sep='\t', index_col=0)

    # Optionally remove rows marked as "undef" in the transcript column
    if remove_undef and (transcript_id_label in df.columns):
        df = df[df[transcript_id_label] != "undef"]

    # Reset and transpose so columns become features (var) and rows become observations (obs)
    df = df.reset_index()
    df = df.transpose()

    # Identify potential metadata rows (e.g., geneId, transcriptId, nbExons)
    known_metadata_labels = [gene_id_label, transcript_id_label]
    metadata_rows = [idx for idx in df.index if idx in known_metadata_labels or "Exons" in idx]

    # Extract metadata if present
    metadata_df = df.loc[metadata_rows] if metadata_rows else pd.DataFrame()
    # Drop them from the numeric data
    df = df.drop(metadata_rows, errors='ignore')

    # Convert to float and optionally to a sparse matrix
    try:
        numeric_data = df.values.astype('float32')
    except ValueError:
        print("Error: Non-numeric data present in the DataFrame. Cannot convert to float.")
        return None

    if sparse:
        numeric_data = csr_matrix(numeric_data)

    # Create AnnData object
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        adata = sc.AnnData(
            X=numeric_data,
            obs=pd.DataFrame(index=df.index.astype(str)),
            var=pd.DataFrame(index=df.columns.astype(str))
        )

    # Attach metadata to the AnnData var
    if not metadata_df.empty:
        for row_label in metadata_df.index.unique():
            adata.var[row_label] = metadata_df.loc[row_label].astype(str).values

    # Make observation names unique
    adata.obs_names_make_unique()

    return adata


In [8]:

file_path = get_resource_path("e18.mouse.clusters.csv")
df = pd.read_csv(file_path)
df['barcode'] = df.index.str.split('_').str[1]


mouse_data_str_1 = download_test_data(output_filename='mouse_1.txt')
print("Test data downloaded successfully")


mouse_data_str_2 = download_test_data("https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748089/suppl/GSM3748089%5F951c.isoforms.matrix.txt.gz", output_filename='mouse_2.txt')
print("Test data downloaded successfully")


mouse_1 = read_sicelore_isomatrix(file_path=mouse_data_str_1)
mouse_2 = read_sicelore_isomatrix(file_path=mouse_data_str_2)

combined_mouse_data = iso_concat([mouse_1, mouse_2], batch_type='numeric')


combined_mouse_data.obs_names_make_unique()
# Step 1: Remove any duplicate barcodes in the DataFrame
df_unique = df.drop_duplicates(subset='barcode')

# Step 2: Filter the DataFrame to include only the barcodes present in the AnnData object
df_filtered = df_unique[df_unique['barcode'].isin(combined_mouse_data.obs_names)]

# Step 3: Set the index of the filtered DataFrame to 'barcode' to make the merge easier
df_filtered.set_index('barcode', inplace=True)

# Step 4: Create a DataFrame from the obs DataFrame of the AnnData object to ensure the same index
obs_df = combined_mouse_data.obs.copy()

# Step 5: Initialize a new column 'cell_type' with NaN values in the obs DataFrame
obs_df['cell_type'] = pd.NA

# Step 6: Update the 'cell_type' column with values from the filtered DataFrame where indices match
obs_df.update(df_filtered['illumina.ident'].rename('cell_type'))

# Step 7: Ensure the index is unique and assign the updated DataFrame back to the obs attribute of the AnnData object
if obs_df.index.is_unique:
    combined_mouse_data.obs = obs_df
else:
    raise ValueError("The index of the obs DataFrame is not unique.")

# Now, the 'cell_type' column should be added to the obs DataFrame of your AnnData object
combined_mouse_data = combined_mouse_data[~combined_mouse_data.obs['cell_type'].isna()]


🔎 Looking for file at: /data/analysis/data_mcandrew/allos_2/allos/resources/e18.mouse.clusters.csv
✅ File found at: /data/analysis/data_mcandrew/allos_2/allos/resources/e18.mouse.clusters.csv
Starting download of test data from https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748087/suppl/GSM3748087%5F190c.isoforms.matrix.txt.gz
File already exists: ../data/mouse_1.txt
Test data downloaded successfully
Starting download of test data from https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3748nnn/GSM3748089/suppl/GSM3748089%5F951c.isoforms.matrix.txt.gz
File already exists: ../data/mouse_2.txt
Test data downloaded successfully


/data/analysis/data_mcandrew/Allos_new/allos_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [9]:
mouse_data_str_1

'/data/analysis/data_mcandrew/allos_2/allos/data/mouse_1.txt'

In [10]:
combined_mouse_data

View of AnnData object with n_obs × n_vars = 1109 × 31986
    obs: 'batch', 'cell_type'
    var: 'geneId'

In [11]:
#| hide
from nbdev.showdoc import *

In [12]:
#| hide
import nbdev; nbdev.nbdev_export()